# 3. Recurrent Neural Network and Language Modeling

<br>

## 강의 소개

- 자연어 처리 분야에서 **Recurrent Neural Network(RNN)**를 활용하는 다양한 방법과 이를 이용한 **Language Model**을 학습합니다.
- RNN은 단어간 순서를 가진 문장을 표현하기 위해 자주 사용되어 왔습니다. 이러한 RNN 구조를 활용해 다양한 NLP 문제를 정의하고 학습하는 방법을 소개합니다.
- Language Model은 이전에 등장한 단어를 condition으로 다음에 등장할 단어를 예측하는 모델입니다. 이전에 등장한 단어는 이전에 학습했던 다양한 neural network 알고리즘을 이용해 표현될 수 있습니다. 이번 시간에는 RNN을 이용한 character-level의 language model에 대해서 알아봅니다.
- RNN을 이용한 Language Model에서 생길 수 있는 초반 time step의 정보를 전달하기 어려운 점, gradient vanishing/exploding을 해결하기 위한 방법 등에 대해 다시 한번 복습할 수 있는 시간이 됐으면 합니다.

<br>

## Further Reading

- [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [CS231n(2017)_Lecture10_RNN](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf)

<br>

## 3.1 Basics of Recurrent Neural Networks (RNNs)

- Basic problem settings
- Model architecture and how it works

<br>

### 3.1.1 Types of RNNs

- Basic structure
  - 왼쪽 그림 : rolled diagram
  - 오른쪽 그림 : unrolled diagram

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1aOdOP371p2xZnieXfGhkq_7anQrzxoaB' width=600/>

- 출처: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

<br>

### 3.1.2 Recurrent Neural Network

#### 3.1.2.1 Inputs and outputs of RNNs (rolled version)

- We usually want to predict a vector at some time steps
- RNN 에서 나온 hidden state vector는 다음 time step 의 입력으로 사용됨과 동시에 필요한 경우 출력값으로 계산될 수 있어야 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1gZYFxKj_HzFfvZuiEvnNGp3tHIVIBnZe' width=200/>

- 출처: http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

<br>

#### 3.1.2.2 How to calculate the hidden state of RNNs

- We can process a sequence of vectors by applying a recurrence formula at every time step

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=18pARO--xmrKlp0T_DZwMDULjKBcArUiG' width=400/>

- 출처: http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

<br>

**RNN 의 구성 요소들**

- $h_{t-1}$
  - old hidden-state vector
  - 전 time step 인 t-1 에서 계산된 hidden state vector
- $x_t$
  - input vector at some time step
  - 현재 time step t 에서의 입력 벡터
- $h_t$
  - new hidden-state vector
  - $h_{t-1}$ 와 $x_t$ 를 입력으로 받아서 계산되는 현재 time step t 에서의 hidden state vector
- $f_W$
  - RNN function with parameters $W$
  - $W$ 를 파라미터로 갖는 RNN 함수
- $y_t$
  - output vector at time step $t$
  - $h_t$ 를 바탕으로 계산되는 현재 time step t 에서의 출력
  - $y_t$ 는 매 time step 마다 계산될 수도 있고 안될수도 있다.

<br>

**RNN 의 주요 특징: 파라미터 공유**

- <font color='red'>Notice: The same function and the same set of parameters are used at every time step</font>
- 매 time step 마다 RNN 을 정의하는 파라미터 $W$ 는 모든 time step 에서 동일한 값을 공유한다는 것이 RNN 의 가장 큰 특징이다.

<br>

**RNN 의 구성요소 계산 방법**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1yxXygVj1l2v51lA3hJHFfLc-LFCI-jU4' width=600/>

- The state consists of a single "hidden" vector **h**
- $f_W$ 함수를 정의해보자.
- $x_t$ 가 3차원 벡터로 주어진 상황을 생각해보자.
- $h_{t-1}$ 이 2차원의 형태로 입력으로 들어온다고 하자.
  - hidden state 의 차원 수는 하이퍼파라미터이다.
- 3차원 벡터 $x_t$ 와 2차원 벡터 $h_{t-1}$ 을 결합해서 하나의 fully connected layer 로 구성된 RNN 모듈을 생각해볼 수 있다.
- 총 5개의 노드로 구성된 입력 layer 를 통해서 $h_t$ 를 계산한다.
  - $h_t$ 의 차원의 경우에도 $h_{t-1}$ 과 동일한 형태의 차원(2)을 가져야 한다.
- 이렇게 구한 $h_t$ 에 non-linear function 인 tanh 를 적용해줌으로서 최종적인 $h_t$ 를 얻게 된다.



<br>

**$f_W \left(h_{t-1}, x_t\right) \rightarrow \left(W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1}\right)$ 식 유도하기**

- 위의 fully connected layer 의 가중치를 $W$ 라고 정의하면 다음과 같은 수식을 정의할 수 있다.
  - $h_t = W \cdot \text{concatenate}(x_t, h_{t-1})$
  - 2x1 = 2x5 $\cdot$ 5x1(3x1, 2x1)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1mpXwMKTd9d_tgGoupdu9mcIyEwl2Mwgo' width=400/>

<br>

- $h_t$ 의 첫 번째 노드를 계산할 때 필요한 연산을 생각해보면 다음과 같이 내적을 수행할 것이다. (초록색 굵은 선)
- 이 때 $W$ 의 3차원 벡터와 $x_t$ 를 내적한 결과(빨간색 선)와 $W$ 의 나머지 2차원 벡터와 $h_{t-1}$ 을 내적한 결과(초록색 선)를 더해주면 $h_t$ 의 첫 번째 노드의 값을 계산할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1U_iYRAA2A4Y7JsJAbpfE6H2TCDMRGNaS' width=400/>

- $h_t$ 의 두 번째 노드도 같은 방법으로 계산할 수 있다.

<br>

- 그렇다면 우리는 $W$ 를 $W_{xh}$ 와 $W_{hh}$ 로 나눠서 생각해볼 수 있다. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11J35R0g5OIC51gGx_9eS-bv79j-NvfzT' width=400/>

- 이를 활용하여 $h_t$ 를 다음과 같은 수식으로 표현할 수 있게 된다.
  - $h_t = W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1}$
- $W_{xh}$ 는 $x_t$ 를 $h_t$ 로 변환해주는 역할을 수행하는 행렬이다.
- $W_{hh}$ 는 $h_{t-1}$ 를 $h_t$ 로 변환해주는 역할을 수행하는 행렬이다.

<br>

**$y_t$ 계산**

- $y_t$ 는 추가적인 output layer 를 만들어서 $h_t$ 에 linear transformation matrix 인 $W_{hy}$ 를 곱해서 계산할 수 있다.
  - $W_{hy}$ 는 $h_t$ 를 $y_t$ 로 변환해주는 역할을 수행하는 행렬이다.
- 이렇게 얻어진 output vector $y_t$ 는 목적에 따라 형태를 변환시켜줄 수 있다.
  - binary classification
    - sigmoid 함수를 적용하여 1차원 스칼라값
  - multi-class classification
    - softmax layer 를 통과해서 확률분포값을  갖는 분류하고자 하는 클래스의 수와 동일한 차원의 벡터를 얻을 수 있다.

<br>

## 3.2 Types of RNNs

- one-to-one
- one-to-many
- many-to-one
- many-to-many

<br>

### 3.2.1 One-to-one

- Standard Neural Network
  - 입력 벡터가 sequential 하지 않은 경우
  - 입력과 출력 어디에도 sequential data 가 아닌 일반적인 형태의 데이터가 사용된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=18oOhCkyspsNRtPYi-nOC-uHr1FLGbUoa' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

### 3.2.2 One-to-many

- Image Captioning
  - 입력으로 하나의 이미지를 제공
  - 이미지에 대한 설명 문구를 생성하는 데 필요한 단어들을 각 time step 별로 출력하는 구조
- One-to-many 는 첫 번째 time step 에만 입력이 들어가는 것을 볼 수 있다.
  - 첫 번째를 제외한 나머지 time step 에 입력으로 줄 데이터가 없을 경우에는 첫 번째 time step 의 입력과 동일한 크기를 갖고 값이 모두 0으로 채워진 벡터, 행렬 또는 텐서가 들어가게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Cc9GRQ_tuVQl3dr7GN3p-2jXBWtpFyhB' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

### 3.2.3 Many-to-one

- Sentiment Classification
  - 문장 안에 단어들을 word embedding 시킨 벡터들을 각 time step 에서 입력으로 받음
  - 해당 문장이 긍부정인 지를 분류하는 하나의 출력을 얻음
- 입력으로 서로 길이가 다른 문장이 들어올 때 길이가 달라진 만큼 RNN cell 이 길이에 맞춰서 확장되어 반복적으로 수행이 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1gNB3KAuuMLPqp10uIxBJC7Oj9EfZdn0d' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

### 3.2.4 Sequence-to-sequence

- 입력과 출력이 모두 sequential 한 형태를 가지는 구조
- Machine Translation
  - 입력으로 들어오는 영어 문장을 읽은 후 마지막 time step 에서부터 한글 문장을 출력하는 구조
  - 입력 단어가 3개이고 출력 단어가 3개인 경우, time step 은 총 5개가 된다.
  - **입력되는 문장을 다 읽은 후 예측 단어를 출력한다는 것이 특징**이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Bz14AV8Ykhn842Y8nbCFGX5ghf0dkpYw' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

- many-to-many 의 또 다른 형태로는 입력을 다 읽은 후 출력을 생성하는 것이 아니라 입력이 주어질 때마다 예측을 수행하는 형태도 존재한다.
- 실시간성이 요구될 때 사용되는 구조
- Video classification on frame level
  - 입력으로 동영상의 각 이미지 프레임이 들어올 때 해당 프레임이 어떤 scene 에 해당하는 지를 예측하는 구조
- 단어별로 품사를 예측하는 task

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1F0p2mwS3Zirc2J2wEfsnS-2vP2XxfWYo' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

## 3.3 Character-level Language Model

### 3.3.1 Language Model task

- 주어진 문자열이나 단어들의 순서를 바탕으로 다음 단어가 무엇인 지를 예측하는 task
- word level, character level 에서 모두 수행할 수 있다.


<br>

### 3.3.2 Example of training sequence "hello"

- 이번 예제에서는 좀 더 simple 한 character level 의 예제를 살펴본다.
- Example training sequence: "hello"
  - 학습 단어로 하나의 단어인 "hello" 만 존재한다고 가정한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1bZbpo1RaokEwqvyugOu3vdxFE1gt7Pxr' width=200/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

#### 3.3.2.1 단어 사전 구축 및 one-hot vector 생성

- 주어진 데이터를 바탕으로 가장 먼저 사전을 구축한다.
  - Vocabulary : `[h, e, l, o]`
- 사전 안의 각 단어들은 사전의 크기만큼의 차원을 갖는 one-hot vector 로 변환된다.
  - `h = [1 0 0 0]`
  - `e = [0 1 0 0]`
  - `l = [0 0 1 0]`
  - `o = [0 0 0 1]`




<br>

#### 3.3.2.2 input layer

- 주어진 character 의 sequence 가 주어질 때 첫 번째 time step 에서는 입력으로 들어오는 `h` 다음에 등장할 단어인 `e` 를 예측해야 한다.
- 두 번째 time step 에서는 입력으로 `h`, `e` 가 주어지면 다음 단어인 `l` 을 예측해야 한다.
- 각각의 단어들의 각 time step 에서 one-hot vector 형태로 입력으로 주어진다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=129u9LbxRuyd-cuRj-TFOG9q5gLmilAGr' width=600/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

#### 3.3.2.3 hidden layer

$\quad h_{t}=\tanh \left(W_{h h} h_{t-1}+W_{x h} x_{t}+b\right)$

<br>

- 입력으로 주어진 one-hot vector 를 RNN 모듈은 현재 time step 에서 들어오는 입력 벡터와 이전 time step 에서 들어오는 hidden state vector 를 선형 결합을 해서 얻어지는 $h_t$ 를 만들어낸다. ($W_{h h} h_{t-1}+W_{x h} x_{t}+b$)
  - 이와 같은 수식으로 $h_t$ 를 구하게 되면 fully connected layer 의 구조로서 RNN 모듈을 생각할 수 있다.
- 마지막으로 비선형 변환인 tanh 를 통과한 후 최종적인 $h_t$ 를 얻어낼 수 있다.
- 첫 번째 time step 에서 입력으로 사용되는 $h_0$ 가 필요하게 되는 데 이는 값이 모두 0으로 이루어진 벡터를 RNN 의 입력으로 줘야 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1FjNo5ZcpktNaT9aCe0ZL0y0vJ6SaC5Kc' width=600/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

#### 3.3.2.4 output layer

- 각 time step 마다 다음에 나올 단어를 예측해야 한다.
  - 이는 many-to-many task 에 해당한다.
- output vector 를 계산하기 위해서 해당 time step 에서 구해진 $h_t$ 를 output layer 를 적용해서 최종 output 을 얻어내게 된다.
  - Logit $=W_{h y} h_{t}+b$
  - 이 값이 **Logit** 이라고 표현된 이유는 사전에서 정의된 4개의 character 중 하나의 character 를 다음에 나올 character 로 예측하는 task 이기 때문이다.
  - 그러므로 output vector 의 차원은 사전의 크기와 동일한 값을 갖게 된다.
- 이렇게 얻어진 output vector 에 multi-class classification 을 수행하기 위해 softmax layer 를 통과시켜서 확률분포로 변환한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1AH60fvx37FMiSR7bFRvs8eJetYKrJyj7' width=600/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

#### 3.3.2.5 inference

- At test-time, sample characters one at a time, feed back to model
- 학습을 끝낸 후 inference 를 수행할 때를 생각해보자.
- 이 때 첫 번째 time step 에서 character `h` 만을 입력으로 주고 여기서 얻어진 예측된 character 를 다음 time step 의 입력값으로 재사용한다.
- 이를 통해 무한한 길이의 sequential character 를 자유롭게 생성할 수 있게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16sK1nd4ExPtVyY3Yhlkk5oNcKBQIEXB3' width=500/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

### 3.3.3 Training a RNN on Shakespeare's plays

- Language Model 을 문단에 대해서도 학습시킬 수 있다.
- 아래의 문단은 셰익스피어의 희곡의 한 문단이다.
- 이 문단에서 나타나는 공백 문자, 쉼표, 줄바꿈 등도 character 로서 사전에 등록되어 활용된다.
  - 이렇게 하면 전체 문단을 one-dimensional character sequence 로 볼 수 있다.
  - 이를 통해 language model 을 학습할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ZB3KU5CZiDwbydfzVwbxsSZJ82Whu6Al' width=800/>

- 출처: http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

<br>

#### 3.3.3.1 Training process of RNN

- 학습이 진행될수록 첫 번째 character 만 줬을 때 만들어내는 문장이 정확해지는 것을 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=12Y5Zw3fa46w9ck4N8D57_ty-6U5ZPuXS' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

### 3.3.4 Results of trained RNN

- 왼쪽에 있는 것과 같은 각 등장 인물별로 대사를 기록한 글을 RNN에 학습시킬 수도 있다.
- 이 데이터를 통해 학습한 RNN 모델은 오른쪽과 같인 등장인물별 대사를 생성해내는 것을 볼 수 있다.
  - 사람 이름이 등장할 때는 연속적인 대문자 사용
  - 사람 이름이 끝났을 때는 ":" 을 붙여줌
  - ":" 이 나온 다음에는 줄바꿈을 실시


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Sdthdp-UIoHjRkB-fb-Zn3bEiYVwRXLr' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

### 3.3.5 A paper written by RNN

- 또 다른 예시로서 논문을 생성하는 모델도 만들 수 있다.
- 논문이 latex 으로 작성되어 있기 때문에 이를 RNN 모델이 학습하게 된ㄷ다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=19KacTpIbGgsF8o6td04Rdp0v_bmJukOT' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

### 3.3.6 C code generated by RNN

- C 언어로 짜여진 프로그래밍 코드를 생성할 수도 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1aKFM0uTEExAGA9pPqLeq1lWDSJnloam3' width=600/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

## 3.4 Backpropagation through time (BPTT)

### 3.4.1 문제점

-  Forward through entire sequence to compute loss, then backward through entire sequence to comput gradient
- loss function 을 통해서 전체 네트워크가 학습된다.
- 다음과 같은 가중치 행렬들이 backpropagation 에 의해 학습이 진행된다.
  - $W_{xh}$: 입력 벡터가 hidden state vector 로 변환될 때 사용되는 가중치
  - $W_{hh}$: 이전 time step 의 hidden state vector 가 현재 time step 의 hidden state vector 로 변환될 때 사용되는 가중치
  - $W_{hy}$: hidden state vector 가 output vector 로 변환될 때 사용되는 가중치
- 한 문단, 즉 character 의 전체 sequence 가 매우 긴 학습 데이터가 주어졌을 때에는 GPU를 통해서 한 번에 학습해야 하는 sequence 도 매우 길어지게 되서 GPU 메모리에 한꺼번에 담기지 못할 수 있다.
- 이러한 문제를 해결하기 위해 truncation 기법을 사용한다.
  - 제한된 길이의 sequence 만으로 학습을 진행 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1bfxYWcMqkxZMgHTASVfHepICB3OUmwtx' width=800/>

- 출처: http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

<br>

### 3.4.2 해결 방안

- Run forward and backward through chunks of the sequence instead of whole sequence
- 한 번에 학습될 수 있는 제한된 sequence 의 길이를 아래 그림과 같이 7개로 한정한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ZGBaITgQ74exnh1zw6qLy5VByrFIx8o7' width=400/>

- 출처: http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

<br>

-  Carry hidden states forward in time forever, but only backpropagate for some smaller number of steps
- 전체 학습 데이터는 굉장히 길지만 전체 sequence 를 7개씩 나눠서 이어 붙임으로서 RNN 의 파라미터들을 학습시킬 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1T-XJcKh_lGU-6j5N5QFxt874jIzcHSnu' width=600/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=190ZscKhfFmOMbA-83ma9dBiw0sX_JmlI' width=800/>

- 출처: http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

<br>

## 3.5 Searching for Interpretable(해석할 수 있는) Cells

- RNN 이 위에서 소개된 데이터를 학습하는 데 필요한 지식을 어떻게 배울 수 있었는 지에 대한 정보가 RNN 내의 어느 부분에 저장되었는 지를 다양한 방식으로 분석할 수 있다.
- RNN 에서 필요한 정보를 저장하는 공간은 매 time step 마다 업데이트를 수행하는 <font color='yellow'>$h_t$, hidden state vector 라고 할 수 있다.</font>
- 만약 hidden state vector 가 3차원이라고 한다면 3개의 숫자 중 어디에 저장되었는 가를 역추적하는 방식으로 분석을 수행할 수 있다.
  - hidden state vector 의 하나의 차원의 값을 고정하고 time step 이 진행됨에 따라서 어떻게 변화하는 지를 분석함으로서 RNN 의 특성을 분석할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1XAnASicbn1AgTUtPBy_GYJ99O6Kjz9BE' width=800/>

- 출처: http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf

<br>

### 3.5.1 How RNN works

- 아래 그림은 특정한 hidden state 의 값을 고정해 놓고 해당 dimension 의 값이 어떻게 변하는 지를 관찰한 것이다.
  - 빨간색: 진할수록 큰 양수 값
  - 파란색: 진할수록 큰 음수 값

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Tx-6pwNWN9wFgg9k3y4FcwLYJykeLK2n' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

#### 3.5.1.1 Quote(따옴표) detection cell

- 그렇게 해봤을 때 여러 dimension 중에 특정 dimension 에서는 아래 그림과 같이 흥미로운 패턴을 보이는 것을 발견할 수 있다.
  - 첫 번째 문장에서 따옴표가 열리고 닫히는 동안 값이 음수(파란색)로 유지된다.
  - 따옴표가 닫힌 이후엔 값이 양수(빨간색)로 유지되다가 따옴표가 다시 열린 다음부턴 음수(파란색)을 나타내는 것을 볼 수 있다.
- 이것이 RNN 내에서 hidden state vector 의 특정 dimension 이 하는 역할을 알려주고 있다.
  - hidden state vector 의 dimension 의 역할이 바로 따옴표가 열렸거나 닫힌 상태를 기억하는 것이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16lNVZtY0VsK6UyybEYiR7jbc56XOpBpy' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

#### 3.5.1.2 `If` statement cell

- 비슷한 사례로 또 다른 dimension 의 경우 프로그래밍 언어에서 if 라는 조건문에 명시되는 조건이 빨간색으로 표시되는 것을 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1QXx0_psBhmHz5yMdMX2WEUwU-4sWp2gd' width=800/>

- 출처: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<br>

## 3.6 Vanishing/Exploding Gradient Problem in RNN

### 3.6.1 RNN is excellent, but...

-  Multiplying the same matrix at each time step during backpropagation causes gradient vanishing or exploding
- 심플한 형태의 fully connected layer 로 구성되는 Vanilla RNN 을 많이 사용하지 않는다.
- 그 이유는 Vanilla RNN 에서는 동일한 matrix 를 매 time step 마다 곱하게 된다. (아래 그림 참고)
- 이 경우 $y_{t+1}$ 에서 필요한 정보를 갖고 있는 멀리 있는 이전 time step 에 있는 hidden state 를 제대로 학습시키려면 backpropagation 이 제대로 이루어져야 한다.
- 하지만 <font color='yellow'>$W_{hh}$ 가 반복적으로 곱해져서 적용된다는 사실 때문</font>에 backpropagation 시 $y_{t+1}$ 에서 발생한 gradient 가 기하급수적으로 커지거나(exploding) 기하급수적으로 작아지는(vanishing) 현상이 발생하게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=18tnG9nsstzH-Fsj_ZnnlAWrLkTn6_R8J' width=600/>

<br>

### 3.6.2 Toy Example

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1PqEen5ut9S8SVxulqA3Yi3PWTJ24PUoJ' width=400/>

- $h_{t}=\tanh \left(w_{x h} x_{t}+w_{h h} h_{t-1}+b\right), t=1,2,3$
  - For $w_{h h}=3, w_{x h}=2, b=1$
  - $x_t$ 와 $h_{t-1}$ 이 각각 스칼라 값이라고 생각해보자.

<br>

- 아래 수식과 같이 동일한 구조 안에서 $h_1$ 에서 $h_3$ 까지 값이 변환되게 되고 $h_3$ 은 하나의 식으로 표현될 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1-hfq2K78YnX8IB5zsn-5793ksarLqnAa' width=400/>

- $y$ 에서 gradient 가 발생됐다고 했을 때 이 gradient 가 $h_3$을 통해 $h_1$ 까지 전달되는 과정을 살펴보면 $h_3$에 대한 $h_1$ 의 편미분 값을 계산하게 된다.
- 이 때 tanh 함수의 미분값을 반복해서 구하다 보면 $W_{hh}$ 의 값인 3이 계속해서 곱해지게 된다.
- 이렇게 되면 time step 갯수 만큼 $W_{hh}$ 의 값 3이 거듭제곱하여 곱해지게 되면서 gradient 값이 증폭하게 되는 것을 볼 수 있다.

<br>

### 3.6.3 Vanishing Gradient Problem in RNN

- The reason why the vanishing gradient problem is important
- 위쪽의 숫자는 time step 이 앞쪽으로 거슬러 올라가는 것을 나타낸다.
- 각각의 정사각형은 $W_{hh}$ 가 정사각 행렬의 모양을 갖기 때문에 정사각 행렬에서 발생되는 gradient 가 변하는 모습을 나타낸 것이다.
- 진한 회색일 경우 값이 0에 가깝다는 것을 의미한다.
- RNN 의 경우 약간의 이전 time step 으로만 가도 gradient 가 0으로 수렴하지만 LSTM 의 경우 꽤 먼 time step 까지도 gradient 가 적절하게 변하는 것을 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1oj3D0RxDMc405itBOM3tuZrfWtGUqxgf' width=800/>

- 출처: https://imgur.com/gallery/vaNahKE

<br>

## 3.7 실습: RNN

1. 주어진 데이터를 RNN에 넣을 수 있는 형태로 만듭니다.
2. 기본적인 RNN 사용법 및 적용법을 익힙니다.
3. PackedSquence의 필요성에 대해 배우고 적용법을 실습합니다.

<br>

### 3.7.1 필요 패키지 import

In [1]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch

<br>

### 3.7.2 데이터 전처리

- 아래의 sample data를 확인해봅시다.  
- 전체 단어 수와 pad token의 id도 아래와 같습니다.

In [2]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [12]:
len(data)

10

In [10]:
min([min(v) for v in data]), max([max(v) for v in data])

(0, 99)

<br>

- Padding 처리를 해주면서 padding 전 길이도 저장합니다.

In [4]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
    valid_lens.append(len(seq))
    if len(seq) < max_len:
        data[i] = seq + [pad_id] * (max_len - len(seq))

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 11052.18it/s]


In [5]:
for d in data:
    print(d)

[85, 14, 80, 34, 99, 20, 31, 65, 53, 86, 3, 58, 30, 4, 11, 6, 50, 71, 74, 13]
[62, 76, 79, 66, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[93, 77, 16, 67, 46, 74, 24, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[19, 83, 88, 22, 57, 40, 75, 82, 4, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[70, 28, 30, 24, 76, 84, 92, 76, 77, 51, 7, 20, 82, 94, 57, 0, 0, 0, 0, 0]
[58, 13, 40, 61, 88, 18, 92, 89, 8, 14, 61, 67, 49, 59, 45, 12, 47, 5, 0, 0]
[22, 5, 21, 84, 39, 6, 9, 84, 36, 59, 32, 30, 69, 70, 82, 56, 1, 0, 0, 0]
[94, 21, 79, 24, 3, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[80, 80, 33, 63, 34, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[87, 32, 79, 65, 2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80, 0, 0]


In [7]:
print(valid_lens)

[20, 5, 8, 10, 15, 18, 17, 6, 6, 18]


<br>

- 위 데이터를 하나의 batch 로 만들어 실습에 이용하겠습니다.

In [8]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data) # (B, L)
batch_lens = torch.LongTensor(valid_lens) # (B,)

<br>

### 3.7.3 RNN 사용해보기

- RNN 에 넣기 전 word embedding 을 위한 embedding layer 를 만듭니다.

In [11]:
embedding_size = 256
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size)

# d_w: embedding size
batch_emb = embedding(batch) # (B, L, d_w)
print(batch_emb.size())

torch.Size([10, 20, 256])


<br>

- 아래와 같이 RNN 모델 및 초기 hidden state를 정의합니다.

In [14]:
hidden_size = 512  # RNN의 hidden size
num_layers = 1  # 쌓을 RNN layer의 개수
num_dirs = 1  # 1: 단방향 RNN, 2: 양방향 RNN

rnn = nn.RNN(
    input_size=embedding_size, # 임베딩된 단어의 차원
    hidden_size=hidden_size, # hidden state 의 차원
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size)) # (num_layers * num_dirs, B, d_h)
print(h_0.size())

torch.Size([1, 10, 512])


<br>

- RNN 에 batch data 를 넣으면 아래와 같이 2가지 output 을 얻는다.
  - `hidden_states`: 각 time step 에 해당하는 hidden state 들의 묶음
  - `h_n`: 모든 sequence 를 거치고 나온 마지막 hidden state

In [16]:
print(batch_emb.size())
print(batch_emb.transpose(0,1).size())

torch.Size([10, 20, 256])
torch.Size([20, 10, 256])


In [17]:
hidden_states, h_n = rnn(batch_emb.transpose(0, 1), h_0)

# d_h: hidden size, num_layers: layer 개수, num_dirs: 방향의 개수
print(hidden_states.shape)  # (L, B, d_h)
print(h_n.shape)  # (num_layers*num_dirs, B, d_h) = (1, B, d_h)

torch.Size([20, 10, 512])
torch.Size([1, 10, 512])


<br>

### 3.7.4 RNN 활용법

- 마지막 hidden state 를 이용하여 text classification task 에 적용할 수 있습니다.

In [18]:
num_classes = 2
classification_layer = nn.Linear(hidden_size, num_classes)

# C: number of classes
output = classification_layer(h_n.squeeze(0)) # (1, B, d_h) -> (B, C)
print(output.shape)

torch.Size([10, 2])


<br>

- 각 time step 에 대한 hidden state 를 이용하여 token-level 의 task 를 수행할 수도 있습니다.

In [19]:
num_classes = 5
entity_layer = nn.Linear(hidden_size, num_classes)

# C: number of classes
output = entity_layer(hidden_states) # (L, B, d_h) -> (L, B, C)
print(output.shape)

torch.Size([20, 10, 5])


<br>

### 3.7.5 PackedSequence 사용법

- 앞서 padding 처리했던 데이터를 다시 확인해봅시다.

In [20]:
data

[[85, 14, 80, 34, 99, 20, 31, 65, 53, 86, 3, 58, 30, 4, 11, 6, 50, 71, 74, 13],
 [62, 76, 79, 66, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [93, 77, 16, 67, 46, 74, 24, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [19, 83, 88, 22, 57, 40, 75, 82, 4, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [70, 28, 30, 24, 76, 84, 92, 76, 77, 51, 7, 20, 82, 94, 57, 0, 0, 0, 0, 0],
 [58, 13, 40, 61, 88, 18, 92, 89, 8, 14, 61, 67, 49, 59, 45, 12, 47, 5, 0, 0],
 [22, 5, 21, 84, 39, 6, 9, 84, 36, 59, 32, 30, 69, 70, 82, 56, 1, 0, 0, 0],
 [94, 21, 79, 24, 3, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [80, 80, 33, 63, 34, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [87, 32, 79, 65, 2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80, 0, 0]]

<br>

- 아래 그림과 같이 불필요한 pad 계산이 포함됩니다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1AmyvjeWuEDtzFpuYZ11ItfV3DTe832xs' width=800/>

<br>

- 데이터를 padding 전 원래 길이 기준으로 정렬합니다.

In [21]:
sorted_lens, sorted_idx = batch_lens.sort(descending=True)
sorted_batch = batch[sorted_idx]

In [23]:
print(sorted_batch)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])


In [24]:
print(sorted_lens)

tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


<br>

- 아래와 같은 padding 무시 효과를 얻을 수 있습니다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lz3wvWAD0EkKsOqJ0C9q9MPvOTObfKH1' width=800/>

<br>

- `pack_padded_sequence` 를 이용하여 PackedSequence object 를 사용한다.

In [25]:
sorted_batch_emb = embedding(sorted_batch)
packed_batch = pack_padded_sequence(sorted_batch_emb.transpose(0, 1), sorted_lens)

print(packed_batch)
print(packed_batch[0].shape)

PackedSequence(data=tensor([[ 0.5675,  0.0037, -1.0423,  ...,  0.1366,  0.3375, -0.1567],
        [ 0.3605,  0.4326,  0.6642,  ...,  0.2936, -1.2819, -1.9344],
        [-0.8049, -0.1083, -0.2633,  ...,  1.3228, -0.5326, -0.7870],
        ...,
        [-0.8811, -0.3274,  0.7290,  ...,  2.0362, -1.2792, -1.2598],
        [-0.9903,  0.8745,  2.5449,  ..., -0.0331, -1.5226,  1.4135],
        [-0.6412, -1.4564, -0.5998,  ...,  0.3748, -1.2355,  0.3707]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 256])


In [26]:
packed_outputs, h_n = rnn(packed_batch, h_0)

print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

PackedSequence(data=tensor([[-0.0704,  0.6465,  0.0291,  ..., -0.3865, -0.3754, -0.1105],
        [-0.2697, -0.1752,  0.0622,  ...,  0.2062,  0.7887,  0.1123],
        [-0.1499,  0.1588,  0.2349,  ...,  0.4513, -0.5608, -0.4134],
        ...,
        [ 0.5305, -0.4452,  0.6720,  ...,  0.6249, -0.1087,  0.3764],
        [ 0.3286,  0.0144, -0.7842,  ...,  0.5720,  0.0443,  0.3746],
        [ 0.4228,  0.3772,  0.4790,  ...,  0.6667, -0.1504, -0.6163]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 512])
torch.Size([1, 10, 512])


<br>

- `packed_output`은 PackedSquence이므로 원래 output 형태와 다릅니다.  
- 이를 다시 원래 형태로 바꿔주기 위해 `pad_packed_sequence`를 이용합니다.

In [27]:
outputs, outputs_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, d_h)
print(outputs_lens)

torch.Size([20, 10, 512])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])
